In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subplots
import pandas as pd
import subprocess as sp
import numpy as np
from sklearn.decomposition import PCA
import os

In [ ]:
# set the path to the data
datapath = "~/shape_map_files/"

In [ ]:
def load_map_data(segment,complex,cutoff=-2):
    """
    Load the data from the csv file
    :return: a dataframe
    """
    file = get_map_file(segment,complex)
    df = pd.read_csv(file,sep="\t",header=None, names=["position","reactivity","error","nucleotide"])
    # mark values under cutoff as nan
    df.loc[df["reactivity"]<cutoff,"reactivity"] = np.nan
    # return (df['reactivity'] - np.nanmean(df['reactivity'])) / np.nanstd(df['reactivity'])
    return list(df["reactivity"])

def get_map_file(segment,complex):
    if complex=="uncomplexed":
        d = {
            "s6":f"{datapath}S6/S6-t_215ShM/S6-t-215_FJ969724.1_BTV1-SA_S6.map",
            "s7":f"{datapath}S7/S7-g_215ShM/S7-g-215_FJ969725.1_BTV1-SA_S7.map",
            "s8":f"{datapath}S8/S8-b_215ShM/S8-b-215_FJ969726.1_BTV1-SA_S8.map",
            "s9":f"{datapath}S9/S9-a_215ShM/S9-a-215_FJ969727.1_BTV1-SA_S9.map",
            "s10":f"{datapath}S10/S10-a_215SM/S10-a-215_FJ969728.1_BTV1-SA_S10.map",
        }
        return d[segment]
    else:
        acessions = {
            "s1": "FJ969719.1",
            "s2": "FJ969720.1",
            "s3": "FJ969721.1",
            "s4": "FJ969722.1",
            "s5": "FJ969723.1",
            "s6": "FJ969724.1",
            "s7": "FJ969725.1",
            "s8": "FJ969726.1",
            "s9": "FJ969727.1",
            "s10": "FJ969728.1",
        }
        
        return f"{datapath}/shapemapper_out/{complex.upper()}/shapemapper_out/{complex.upper()}_{acessions[segment]}_{segment.upper()}.map"

def cname(complexes):
    c = {
        "uncomplexed":"Uncomplexed",
        "b10":"s7-9",
        "b9":"s9-10",
        "b8":"s8-10",
        "b7":"s7-10",
        "b6":"s6-10",
        "b5":"s5-10",
        "b4":"s4-10",
        "b3":"s3-10",
        "b2":"s2-10",
        "b1":"s1-10",
        "b12":"s10mut",
    }
    if isinstance(complexes,str):
        return c[complexes]
    else:
        return [c[x] for x in complexes]

def run_delta_shape(segment,complex1,complex2):
    """
    Run deltaSHAPE on the data
    :return:
    """
    file1 = get_map_file(segment,complex1 )
    file2 = get_map_file(segment,complex2 )
    sp.run(f"~/mambaforge/envs/py2/bin/python ~/projects/shape_map/deltaSHAPE/deltaSHAPE_v1.0/deltaSHAPE.py {file2} {file1} --out {datapath}/deltaSHAPE/{segment}_{complex1}_{complex2}.tsv --noshow --all",shell=True)

def load_delta_shape(segment,complex1,complex2):
    """
    Load the data from the csv file
    :return: a dataframe
    """
    file = f"{datapath}/deltaSHAPE/{segment}_{complex1}_{complex2}.tsv"
    if not os.path.isfile(file):
        run_delta_shape(segment=segment,complex1=complex1,complex2=complex2)
    print(file)
    df = pd.read_csv(file,sep="\t")
    # mark values under cutoff as nan
    df.loc[df["DeltaSHAPE"]< -900,"DeltaSHAPE"] = np.nan
    return df

In [ ]:
def plot_heatmap(segment,complexes):
    data = [load_map_data(segment,c) for c in complexes]
    fig = px.imshow(
        data,
        labels=dict(x='Nucleotide position', y='Complex', color='SHAPE reactivity'),
        aspect='auto',
        template='simple_white',
        # color_continuous_scale=[(0, "red"), (0.5, "green"), (1, "blue")]
        color_continuous_scale="RdBu",
        range_color=[-3,3]
    )
    # update yaxis labels
    fig.update_yaxes(
        ticktext=cname(complexes),
        tickvals=list(range(len(complexes)))
    )
    return fig

complexes = ['uncomplexed','b10','b8','b7','b6','b3','b1','b12']

plot_heatmap("s8",complexes)


In [ ]:
def remove_columns_with_nan(matrix):
    # Convert the matrix to a NumPy array
    matrix_array = np.array(matrix)
    
    # Find columns with NaN values
    columns_with_nan = np.isnan(matrix_array).any(axis=0)
    
    # Remove columns with NaN values
    matrix_without_nan = matrix_array[:, ~columns_with_nan]
    
    return matrix_without_nan.tolist()


def run_pca(segment,complexes):
    data = [load_map_data(segment,c) for c in complexes]
    data = remove_columns_with_nan(data)
    pca = PCA(n_components=2)
    pca.fit(data)
    print(pca.explained_variance_ratio_)
    pc = pd.DataFrame(pca.transform(data), columns=['PC1', 'PC2'])
    pc["complex"] = cname(complexes)
    fig = px.scatter(pc, x="PC1", y="PC2", template='simple_white',text="complex")
    # add variance explained to the axis labels
    fig.update_xaxes(title_text=f"PC1 ({round(pca.explained_variance_ratio_[0]*100,2)}%)")
    fig.update_yaxes(title_text=f"PC2 ({round(pca.explained_variance_ratio_[1]*100,2)}%)")
    # position text
    fig.update_traces(textposition='top center')
    return fig

# print(complexes)
# run_pca("s7",complexes)

In [ ]:
df = load_delta_shape("s7","uncomplexed","b10")
df['DeltaSHAPE'].min()

In [ ]:
box_colours = {
    'uncomplexed': '#1f77b4',
    'b10': '#ff7f0f',
    'b7': '#2ba02b',
    'b6': '#d62727',
    'b5': '#9467bd',
    'b4': '#8c564c',
    'b3': '#e377c1',
    'b2': '#7f7f7f',
    'b1': '#bcbd21',
}

def delta_shape_boxplot(segment,complexes):
    data = []
    for i in range(len(complexes)-1):
        df = load_delta_shape(segment,complexes[i],complexes[i+1])   
        data.append(df["DeltaSHAPE"])

    # make boxplot
    fig = go.Figure()
    for i in range(len(data)):
        print(complexes[i])
        fig.add_trace(
            go.Box(
                y=data[i], name=f"{cname(complexes[i])}/{cname(complexes[i+1])}",
                line=dict(color=box_colours[complexes[i]]),
            )
        )

    # update template
    fig.update_layout(template='simple_white')
    return fig
    
complexes = ["uncomplexed","b10","b7","b6","b5","b4","b3","b2","b1","b12"]
delta_shape_boxplot("s7",complexes)

In [ ]:
def plot_uncomplexed_line(segment):
    data = load_map_data(segment,"uncomplexed")
    fig = px.line(
        y=data,
        template='simple_white',
        labels=dict(x='Nucleotide position', y='SHAPE reactivity'),
    )
    fig.update_traces(line_color='#7570b3',showlegend=False)
    return fig

plot_uncomplexed_line("s6")

In [ ]:
def find_run_locations(column,subset_on=None):
    current_value = column.iloc[0]
    run_start = 0
    run_locations = []

    for i, value in enumerate(column):
        if value != current_value:
            run_end = i - 1
            run_locations.append((run_start, run_end, current_value))
            run_start = i
            current_value = value

    if run_start <= len(column) - 1:
        run_locations.append((run_start, len(column) - 1, current_value))

    if subset_on is not None:
        return [r for r in run_locations if r[2]==subset_on]
    else:
        return run_locations


def delta_shape_plot(segment,complexes):
    # df = load_delta_shape(segment,complexes[0],complexes[1])
    # make subplots
    fig = subplots.make_subplots(rows=len(complexes), cols=1, shared_xaxes=True, vertical_spacing=0.02)
    maxval = 0
    minval = 0
    for i in range(len(complexes)-1):
        df = load_delta_shape(segment,complexes[i],complexes[i+1])   
        maxval = max(maxval,df["DeltaSHAPE"].max())
        minval = min(minval,df["DeltaSHAPE"].min())
        # fill space between line and 0 with light grey
        fig.add_trace(go.Scatter(x=df["Nuc"], y=df["DeltaSHAPE"], fill='tozeroy', mode='none',fillcolor='lightgrey'), row=i+1, col=1)
        # add trace with color red  
        fig.add_trace(go.Scatter(x=df["Nuc"], y=df["DeltaSHAPE"], mode='lines', line=dict(color='#a3a3a3')), row=i+1, col=1)
        


        runs = find_run_locations(df["Sig"],subset_on=1)

        for j,r in enumerate(runs):
            dfslice = df[r[0]:r[1]+1].copy()
            dfslice['tmp'] = (dfslice["DeltaSHAPE"]<0).map(int)

            for ri in find_run_locations(dfslice['tmp']):
                dfslice2 = dfslice[ri[0]:ri[1]+1]
                if ri[2]==0:
                    # color="#1b9e77"
                    color = "#e30a09"
                else:
                    # color="#d95f02"
                    color = "#0926e3"
                fig.add_trace(go.Scatter(x=dfslice2["Nuc"], y=dfslice2["DeltaSHAPE"], mode='lines',line=dict(color=color)), row=i+1, col=1)

    # remove legend
    fig.update_layout(showlegend=False)
    # update template
    fig.update_layout(template='simple_white')
    # set max y for all subplots
    fig.update_yaxes(range=[minval,maxval])
    # set xaxis title
    fig.update_xaxes(title_text="Nucleotide position")
    # set plot height
    return fig
    

delta_shape_plot("s10",["uncomplexed","b10","b7","b6","b1","b12"])

In [ ]:
complexes = ["uncomplexed","b10","b7","b6","b3","b1"]
delta_shape_plot("s7",complexes).write_image("s7_delta_shape_plots.svg")
delta_shape_boxplot("s7",complexes).write_image("s7_delta_shape_boxplot.svg")
delta_shape_boxplot("s7",complexes).write_html("s7_delta_shape_boxplot.html")
plot_heatmap("s7",complexes).write_image("s7_heatmap.png")
plot_uncomplexed_line("s7").write_image("s7_uncomplexed_shape.svg")
complexes = ["uncomplexed","b10","b7","b6","b3","b1","b12"]
run_pca("s7",complexes).write_image("s7_pca.svg")

complexes = ["uncomplexed","b10","b7","b6","b3","b1"]
delta_shape_plot("s8",complexes).write_image("s8_delta_shape_plots.svg")
delta_shape_boxplot("s8",complexes).write_image("s8_delta_shape_boxplot.svg")
delta_shape_boxplot("s8",complexes).write_html("s8_delta_shape_boxplot.html")
plot_heatmap("s8",complexes).write_image("s8_heatmap.png")
plot_uncomplexed_line("s8").write_image("s8_uncomplexed_shape.svg")
complexes = ["uncomplexed","b10","b7","b6","b3","b1","b12"]
run_pca("s8",complexes).write_image("s8_pca.svg")

complexes = ["uncomplexed","b10","b7","b6","b3","b1"]
delta_shape_plot("s9",complexes).write_image("s9_delta_shape_plots.svg")
delta_shape_boxplot("s9",complexes).write_image("s9_delta_shape_boxplot.svg")
delta_shape_boxplot("s9",complexes).write_html("s9_delta_shape_boxplot.html")
plot_heatmap("s9",complexes).write_image("s9_heatmap.png")
plot_uncomplexed_line("s9").write_image("s9_uncomplexed_shape.svg")
complexes = ["uncomplexed","b10","b7","b6","b3","b1","b12"]
run_pca("s9",complexes).write_image("s9_pca.svg")

complexes = ["uncomplexed","b7","b6","b3","b1"]
delta_shape_plot("s10",complexes).write_image("s10_delta_shape_plots.svg")
delta_shape_boxplot("s10",complexes).write_image("s10_delta_shape_boxplot.svg")
delta_shape_boxplot("s10",complexes).write_html("s10_delta_shape_boxplot.html")
plot_heatmap("s10",complexes).write_image("s10_heatmap.png")
plot_uncomplexed_line("s10").write_image("s10_uncomplexed_shape.svg")
complexes = ["uncomplexed","b7","b6","b3","b1","b12"]
run_pca("s10",complexes).write_image("s10_pca.svg")
delta_shape_plot("s10",complexes).write_image("s10_delta_shape_plots_mutant.svg")


complexes = ["uncomplexed","b6","b3","b1"]
delta_shape_plot("s6",complexes).write_image("s6_delta_shape_plots.svg")
delta_shape_boxplot("s6",complexes).write_image("s6_delta_shape_boxplot.svg")
delta_shape_boxplot("s6",complexes).write_html("s6_delta_shape_boxplot.html")
plot_heatmap("s6",complexes).write_image("s6_heatmap.png")
plot_uncomplexed_line("s6").write_image("s6_uncomplexed_shape.svg")
complexes = ["uncomplexed","b6","b3","b1","b12"]
run_pca("s6",complexes).write_image("s6_pca.svg")


In [ ]:

complexes = ["uncomplexed","b10","b7","b12"]
delta_shape_boxplot("s7",complexes).write_image("s7_delta_shape_boxplot_mut.svg")

complexes = ["uncomplexed","b10","b7","b12"]
delta_shape_boxplot("s8",complexes).write_image("s8_delta_shape_boxplot_mut.svg")

complexes = ["uncomplexed","b10","b7","b12"]
delta_shape_boxplot("s9",complexes).write_image("s9_delta_shape_boxplot_mut.svg")

complexes = ["uncomplexed","b7","b12"]
delta_shape_boxplot("s10",complexes).write_image("s10_delta_shape_boxplot_mut.svg")

In [ ]:

complexes = ["uncomplexed","b10","b7","b6","b5","b4","b3","b2","b1","b12"]
delta_shape_boxplot("s7",complexes).write_html("s7_delta_shape_boxplot.html")

complexes = ["uncomplexed","b10","b7","b6","b1","b12"]
delta_shape_boxplot("s8",complexes).write_html("s8_delta_shape_boxplot.html")

complexes = ["uncomplexed","b10","b7","b6","b1","b12"]
delta_shape_boxplot("s9",complexes).write_html("s9_delta_shape_boxplot.html")

complexes = ["uncomplexed","b7","b6","b1","b12"]
delta_shape_boxplot("s10",complexes).write_html("s10_delta_shape_boxplot.html")

In [ ]:
complexes = ["uncomplexed","b10","b7","b6","b5","b4","b3","b2","b1","b12"]
delta_shape_boxplot("s7",complexes).write_html("s7_delta_shape_boxplot.html")

# Running delta shape

In [ ]:
complexes = ["uncomplexed","b10","b7","b6","b5","b4","b3","b2","b1","b12"]
segment = "s7"

for i in range(len(complexes)-1):
    print(complexes[i],complexes[i+1])
    run_delta_shape(segment,complexes[i],complexes[i+1])
# run_delta_shape("s6","uncomplexed","b10")
# run_delta_shape("s6","b10","b7")
# run_delta_shape("s6","b7","b6")
# run_delta_shape("s6","b6","b1")
# run_delta_shape("s6","b1","b12")

# Testing

In [ ]:
data = []
complexes = ["uncomplexed","b10","b7","b6","b1","b12"]
for i in range(len(complexes)-1):
    print(i)
    df = load_delta_shape("s7",complexes[i],complexes[i+1])   
    data.append(df["DeltaSHAPE"])

print(len(data))
# perform anova test
from scipy.stats import f_oneway
import scipy.stats as stats
for i in range(len(complexes)-2):
    print(i,i+1)
    print(complexes[i],complexes[i+1])
    # extract non missing data from each group
    d1 = data[i].dropna()
    d2 = data[i+1].dropna()
    # perform anova test
    f, p = f_oneway(d1, d2)
    # print(f,p)
    # perform f test
    f = np.var(d1, ddof=1)/np.var(d2, ddof=1)
    nun = len(d1)-1
    dun = len(d2)-1
    p = 1-stats.f.cdf(f, nun, dun)
    print(f,p)

# Data concordance

In [ ]:
def load_raghav_map_data(segment,complex,cutoff=-2):
    """
    Load the data from the csv file
    :return: a dataframe
    """
    file = get_raghav_map_file(segment,complex)
    df = pd.read_csv(file,sep="\t",header=None, names=["position","reactivity","error","nucleotide"])
    # mark values under cutoff as nan
    df.loc[df["reactivity"]<cutoff,"reactivity"] = np.nan
    # return (df['reactivity'] - np.nanmean(df['reactivity'])) / np.nanstd(df['reactivity'])
    return list(df["reactivity"])

def get_raghav_map_file(segment,complex):
    if complex=="uncomplexed":
        d = {
            "s6":"/Users/jody/projects/shape_map/SHAPE_raw_data_from_raghav/S6/S6-t_215ShM/S6-t-215_FJ969724.1_BTV1-SA_S6.map",
            "s7":"/Users/jody/projects/shape_map/SHAPE_raw_data_from_raghav/S7/S7-g_215ShM/S7-g-215_FJ969725.1_BTV1-SA_S7.map",
            "s8":"/Users/jody/projects/shape_map/SHAPE_raw_data_from_raghav/S8/S8-b_215ShM/S8-b-215_FJ969726.1_BTV1-SA_S8.map",
            "s9":"/Users/jody/projects/shape_map/SHAPE_raw_data_from_raghav/S9/S9-a_215ShM/S9-a-215_FJ969727.1_BTV1-SA_S9.map",
            "s10":"/Users/jody/projects/shape_map/SHAPE_raw_data_from_raghav/S10/S10-a_215SM/S10-a-215_FJ969728.1_BTV1-SA_S10.map",
        }
        return d[segment]
    else:
        acessions = {
            "s1": "FJ969719.1",
            "s2": "FJ969720.1",
            "s3": "FJ969721.1",
            "s4": "FJ969722.1",
            "s5": "FJ969723.1",
            "s6": "FJ969724.1",
            "s7": "FJ969725.1",
            "s8": "FJ969726.1",
            "s9": "FJ969727.1",
            "s10": "FJ969728.1",
        }
        ld = os.listdir(f"/Users/jody/projects/shape_map/SHAPE_raw_data_from_raghav/{segment.upper()}")
        d = f"/Users/jody/projects/shape_map/SHAPE_raw_data_from_raghav/{segment.upper()}/{ld[0]}"
        mapfile = [x for x in os.listdir(d) if x.endswith(".map") and acessions[segment.lower()] in x][0]
        return f"{d}/{mapfile}"


In [ ]:
for_compaison = [
    ("s7","b10"),
    ("s7","b7"),
    ("s8","b10"),
    ("s8","b7"),
    ("s9","b10"),
    ("s9","b7"),
    ("s10","b7"),
    
    

]
dfs = []
for s,c in for_compaison:

    data1 = load_raghav_map_data(s,c)
    data2 = load_map_data(s,c)
    df = pd.DataFrame({"data1":data1,"data2":data2})
    # remove any rows with nan
    df = df.dropna()
    dfs.append(df)


# calculate r squared
from scipy.stats import linregress
df = pd.concat(dfs)
fit = linregress(df['data1'],df['data2'])
fit.rvalue**2

In [ ]:
dfs

In [ ]:
file1 = '/Users/jody/projects/shape_map/shapemapper_out/B7/shapemapper_out/B7_FJ969728.1_S10.map'
file2 = '/Users/jody/projects/shape_map/SHAPE_raw_data_from_raghav/B7-/B7_FJ969728.1_BTV1-SA_S10.map'

In [ ]:
df1

In [ ]:
newdf = pd.DataFrame({"data1":df1['reactivity'],"data2":df2['reactivity']})
newdf = newdf[(newdf["data1"]>-900) & (newdf["data2"]>-900)]

In [ ]:
fig = px.scatter(x=df['data1'],y=df['data2'],template='simple_white')
# show line with slope of 1 and intercept of 0
fig.add_trace(go.Scatter(x=[0,1],y=[0,1],line=dict(color='black',dash='dash')))
fig.update_layout(showlegend=False)
fig.update_xaxes(title_text="Delta shape 1")
fig.update_yaxes(title_text="Delta shape 2")
fig.write_image("delta_shape_comparison.svg")

# Stable sites with no change

In [ ]:
def calculate_delta_shape_variability(segment,complexes):
    data = {}
    for i in range(len(complexes)-1):
        print(complexes[i],complexes[i+1])
        data[f"{complexes[i]}/{complexes[i+1]}"] = load_delta_shape(segment,complexes[i],complexes[i+1])['DeltaSHAPE']

    df = pd.DataFrame(data)
    df['mean'] = df.mean(axis=1)   
    df['std'] = df.std(axis=1)
    df.to_csv(f'{segment}_delta_shape_variability.csv')

calculate_delta_shape_variability(
    segment = 's10',
    complexes = ["uncomplexed","b7","b6","b3","b1"],
)



In [ ]:
calculate_delta_shape_variability(
    complexes = ["uncomplexed","b10","b7","b6","b3","b1"],
    segment="s7"
)


calculate_delta_shape_variability(
    complexes = ["uncomplexed","b10","b7","b6","b3","b1"],
    segment="s8"
)


calculate_delta_shape_variability(
    complexes = ["uncomplexed","b10","b7","b6","b3","b1"],
    segment="s9"
)


calculate_delta_shape_variability(
    complexes = ["uncomplexed","b7","b6","b3","b1"],
    segment="s10"
)


calculate_delta_shape_variability(
    complexes = ["uncomplexed","b6","b3","b1"],
    segment="s6"
)



In [ ]:
px.line(df.iloc[:,0])